## Time Interaction Fingerprint

In [ ]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionFilter, InteractionFingerprinter
from mmtfPyspark.filters import ContainsAlternativeLocations, NotFilter
import papermill as pm
import time

### Run benchmark

In [ ]:
t0 = time.time()
spark = SparkSession.builder.master("local[" + str(cores) + "]").appName("Fingerprint").getOrCreate()

t1 = time.time()
structures = mmtfReader.read_sequence_file(path)
#structures = structures.filter(NotFilter(ContainsAlternativeLocations()))
                               
interactionFilter = InteractionFilter(distanceCutoff=3.5, minInteractions=1)
interactionFilter.set_query_groups(True, "ASP")
interactionFilter.set_query_atom_names(True, ['OD1','OD2'])
interactionFilter.set_target_groups(True, "ARG")
interactionFilter.set_target_atom_names(True, ['NH1','NH2'])

interactions = InteractionFingerprinter.get_polymer_interactions(structures, interactionFilter)
count = interactions.count()

t2 = time.time()
spark.stop()

### Record results

In [ ]:
pm.record("fingerprint_total", t2 - t0)
pm.record("fingerprint", t2 - t1)